<h1 style="text-align: center;">CNN Segmentation - Neural Network</h1>
<h3 style="text-align: center;">Carlos Moreno</h3>

In [1]:
import os
import torch
import torch.nn as nn
from torch.optim import SGD, Adam
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import numpy as np
import glob
import cv2
import pickle

Run device

In [2]:
if torch.cuda.is_available(): 
    dev = "cuda" 
else: 
    dev = "cpu" 
device = torch.device("cpu")
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 4070 SUPER


->Load Dataset

In [3]:
class MainDataset(Dataset):
    def __init__(self, filepath):
        self.x, self.y = torch.load(filepath) #Upload ·· x - Number, y - Labels
        self.x = self.x / 255. #Normalizamos
        self.y = F.one_hot(self.y, num_classes=10).to(float) # Enconder para clasificación por vectores
    def __len__(self):  
        return self.x.shape[0]
    def __getitem__(self, ix): 
        return self.x[ix], self.y[ix] # i-ésimo numero, label iésima

In [ ]:
class ConvNeuralNet(nn.Module):
    def __init__(self, num_classes = 10):    
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)
        self.R = nn.ReLU()
    def forward(self,x):
        x = self.R(self.conv1(x))
        x = self.pool(x)
        x = self.R(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x.squeeze()

Clase Dataset (Inicializa los datos y los reordena)

In [ ]:
train_ds = MainDataset('..\\Database\\MNIST\\processed\\training.pt')
test_ds = MainDataset('..\\Database\\MNIST\\processed\\test.pt')

train_dl = DataLoader(train_ds, batch_size=5, shuffle=True) # se divide el dataset en batch de a 5
test_dl = DataLoader(test_ds, batch_size=5, shuffle=True) # se divide el dataset en batch de a 5

## Optimización

In [ ]:
def train_model(dl, f, n_epochs=20, device = "cpu"):
    # Optimization
    opt =optimizer = Adam(f.parameters(), lr=3e-4) # karpathy's constant
    L = nn.CrossEntropyLoss()
	
    # Train model
    losses = []
    epochs = []
    accuracy = []
    for epoch in range(n_epochs):
        N = len(dl)
            
        for i, (x, y) in enumerate(dl):
            x, y = x.to(device), y.to(device)
            # Update the weights of the network
            opt.zero_grad() 
            loss_value = L(f(x), y) 
            loss_value.backward() 
            opt.step() 
            # Store training data
            epochs.append(epoch+i/N)
            losses.append(loss_value.item())
            
        print(f"Epoch {epoch + 1} is done; loss = { sum (losses[epoch*i: (epoch + 1)*i]) / 12000 }")
        accuracy.append(CheckAccuracy(train_dl, f, "Train Data")[0])
        
    return np.array(epochs), np.array(losses), accuracy